In [1]:
import numpy as np
import pandas as pd
import requests
import alpha_vantage
from time import *

In [12]:
url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=BRK-B' + alpha_vantage.key
r = requests.get(url)
data = r.json()

print(r)


<Response [200]>


In [4]:
r.status_code

200

In [5]:
data.pop('Meta Data')

{'1. Information': 'Monthly Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'IBM',
 '3. Last Refreshed': '2022-01-14',
 '4. Time Zone': 'US/Eastern'}

In [6]:
df = pd.DataFrame(data['Monthly Time Series']).T
df

,1. open,2. high,3. low,4. close,5. volume
2022-01-14,134.0700,142.2000,127.9700,134.2100,68058451
2021-12-31,118.2500,134.9900,116.5600,133.6600,113930079
2021-11-30,125.0500,127.2900,114.5600,117.1000,119252012
2021-10-29,141.0000,146.0000,124.6200,125.1000,144097758
2021-09-30,139.9800,140.5700,132.7800,138.9300,76644238
...,...,...,...,...,...
2000-04-28,120.0000,128.0000,101.2500,111.5000,168464800
2000-03-31,102.0000,128.2500,99.5000,118.3700,194329000
2000-02-29,112.3700,119.7500,100.9400,102.7500,133524400
2000-01-31,112.4400,124.7500,109.6200,112.2500,175259600


In [7]:
df['Company'] = 'IBM'

In [8]:
df

,1. open,2. high,3. low,4. close,5. volume,Company
2022-01-14,134.0700,142.2000,127.9700,134.2100,68058451,IBM
2021-12-31,118.2500,134.9900,116.5600,133.6600,113930079,IBM
2021-11-30,125.0500,127.2900,114.5600,117.1000,119252012,IBM
2021-10-29,141.0000,146.0000,124.6200,125.1000,144097758,IBM
2021-09-30,139.9800,140.5700,132.7800,138.9300,76644238,IBM
...,...,...,...,...,...,...
2000-04-28,120.0000,128.0000,101.2500,111.5000,168464800,IBM
2000-03-31,102.0000,128.2500,99.5000,118.3700,194329000,IBM
2000-02-29,112.3700,119.7500,100.9400,102.7500,133524400,IBM
2000-01-31,112.4400,124.7500,109.6200,112.2500,175259600,IBM


In [2]:
sp500 = pd.read_csv('sp500.csv')
sp500

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [3]:
symbols = sp500['Symbol']
symbols

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
500     YUM
501    ZBRA
502     ZBH
503    ZION
504     ZTS
Name: Symbol, Length: 505, dtype: object

In [5]:
symbols[symbols == "CFG"]

113    CFG
Name: Symbol, dtype: object

In [22]:
# get income statements for companies
for i, s in enumerate(symbols):
    if i < 500:
        continue
    #if i == 500:
    #    break
    if s == "BRK.B":
        url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=BRK-B' + '&apikey=' + alpha_vantage.key
    elif s == "BF.B":
        url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=BF-B' + '&apikey=' + alpha_vantage.key
    else:
        url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=' + s + '&apikey=' + alpha_vantage.key
    print(s)
    r = requests.get(url)
    if r.status_code != 200:
        print("Unable to get income statements for %s", s)
        print(i)
        continue
    data = r.json()
    df = pd.DataFrame(data['quarterlyReports'])
    df = df.set_index('fiscalDateEnding')
    df['Symbol'] = s
    if i == 0:
        df.to_csv('income_statements.csv')
    else:
        df.to_csv('income_statements.csv', mode = 'a', header = False)
    sleep(12)


YUM
ZBRA
ZBH
ZION
ZTS


In [5]:
r = requests.get('https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=' + s + '&apikey=' + alpha_vantage.key)
print(r.status_code)

200


In [6]:
data = r.json()
data

{'Information': 'Thank you for using Alpha Vantage! You have reached the 500 requests/day limit and there are multiple ways to remove this rate limit: (1) become a holder of Alpha Vantage Coin (AVC), an Ethereum-based cryptocurrency that provides various utility & governance functions within the Alpha Vantage ecosystem (AVC mining guide: https://www.alphatournament.com/avc_mining_guide/) to remove the 500 requests/day rate limit, (2) subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits'}

In [ ]:
# get balance sheets for companies BALANCE_SHEET
for i, s in enumerate(symbols):
    if i < 76:
        continue
    if s == "BRK.B":
        url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=BRK-B' + '&apikey=' + alpha_vantage.key
    elif s == "BF.B":
        url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=BF-B' + '&apikey=' + alpha_vantage.key
    else:
        url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=' + s + '&apikey=' + alpha_vantage.key
    print(s)
    r = requests.get(url)
    if r.status_code != 200:
        print("Unable to get balance sheet for %s", s)
        print(r.status_code)
        break
    data = r.json()
    df = pd.DataFrame(data['quarterlyReports'])
    df = df.set_index('fiscalDateEnding')
    df['Symbol'] = s
    if i == 0:
        df.to_csv('balance_sheets.csv')
    else:
        df.to_csv('balance_sheets.csv', mode = 'a', header = False)
    sleep(15)

In [10]:
# get cash flow for companies CASH_FLOW
for i, s in enumerate(symbols):
    if i < 72:
        continue
    if s == "BRK.B":
        url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol=BRK-B' + '&apikey=' + alpha_vantage.key
    elif s == "BF.B":
        url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol=BF-B' + '&apikey=' + alpha_vantage.key
    else:
        url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol=' + s + '&apikey=' + alpha_vantage.key
    print(s)
    r = requests.get(url)
    if r.status_code != 200:
        print("Unable to get cash flow for %s", s)
        continue
    data = r.json()
    df = pd.DataFrame(data['quarterlyReports'])
    df = df.set_index('fiscalDateEnding')
    df['Symbol'] = s
    if i == 0:
        df.to_csv('cash_flow.csv')
    else:
        df.to_csv('cash_flow.csv', mode = 'a', header = False)
    sleep(15)

BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
CHRW
CDNS
CZR
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CERN
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
COO
CPRT
GLW
CTVA
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DRE
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ENPH
ETR
EOG
EPAM
EFX
EQIX
EQR
ESS
EL
ETSY
EVRG
ES
RE
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FAST
FRT
FDX
FIS
FITB
FE
FRC
FISV
FLT
FMC
F
FTNT
FTV
FBHS
FOXA
FOX
BEN
FCX
GPS
GRMN
IT
GNRC
GD
GE
GIS
GM
GPC
GILD
GL
GPN
GS
GWW
HAL
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUM
HBAN
HII
IEX
IDXX
INFO
ITW
ILMN
INCY
IR
INTC
ICE
IBM
IP
IPG
IFF
INTU
ISRG
IVZ
IPGP
IQV
IRM
JKHY
J
JBHT
SJM
JNJ
JCI
JPM
JNPR
K
KEY
KEYS
KMB
KIM
KMI
KLAC
KHC
KR
LHX
LH
LRCX
LW
LVS
LDOS
LEN
LLY
LNC
LIN
LYV
LKQ
LMT
L
LOW
LUMN
LYB
MTB
MRO
MPC
MKTX
MAR
MMC
MLM
MAS
MA
MTCH
MKC
MCD
MCK
MDT
MRK
FB
MET
MTD
MG

In [ ]:
# get earnings for companies EARNINGS
for i, s in enumerate(symbols):
    if i < 68:
        continue
    if s == "BRK.B":
        url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=BRK-B' + '&apikey=' + alpha_vantage.key
    elif s == "BF.B":
        url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=BF-B' + '&apikey=' + alpha_vantage.key
    else:
        url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=' + s + '&apikey=' + alpha_vantage.key
    print(s)
    r = requests.get(url)
    if r.status_code != 200:
        print("Unable to get earnings for %s", s)
        continue
    data = r.json()
    df = pd.DataFrame(data['quarterlyEarnings'])
    df = df.set_index('fiscalDateEnding')
    df['Symbol'] = s
    if i == 0:
        df.to_csv('earnings.csv')
    else:
        df.to_csv('earnings.csv', mode = 'a', header = False)
    sleep(15)

In [4]:
# get company overview OVERVIEW
for i, s in enumerate(symbols):
    if s == "BRK.B":
        url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=BRK-B' + '&apikey=' + alpha_vantage.key
    elif s == "BF.B":
        url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=BF-B' + '&apikey=' + alpha_vantage.key
    else:
        url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=' + s + '&apikey=' + alpha_vantage.key
    print(s)
    r = requests.get(url)
    if r.status_code != 200:
        print("Unable to get company overview for %s", s)
        continue
    data = r.json()
    df = pd.DataFrame.from_dict(data, orient = 'index').T
    if i == 0:
        df.to_csv('comapany_overview.csv')
    else:
        df.to_csv('company_overview.csv', mode = 'a', header = False)
    sleep(15)

MMM
AOS
ABT
ABBV
ABMD
ACN
ATVI
ADBE
AAP
AES
AFL
A
AIG
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AMD
AEE
AAL
AEP
AXP
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
ANTM
AON
APA
AAPL
AMAT
APTV
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
BKR
BLL
BAC
BBWI
BAX
BDX
BRK.B
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
CHRW
CDNS
CZR
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CERN
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
COO
CPRT
GLW
CTVA
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DRE
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ENPH
ETR
EOG
EPAM
EFX
EQIX
EQR
ESS
EL
ETSY
EVRG
ES
RE
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FAST
FRT
FDX
FIS
FITB
FE
FRC
FISV
FLT
FMC
F
FTNT
FTV
FBHS
FOXA
FOX
BEN
FCX
GPS
GRMN
IT
GNRC
GD
GE
GIS
GM
GPC
GILD
GL
GPN
GS
GWW
HAL
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HP

In [47]:
# get monthly prices
for i, s in enumerate(symbols):
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=' + s + '&apikey=' + alpha_vantage.key
    print(s)
    r = requests.get(url)
    if r.status_code != 200:
        print("Unable to get monthly prices for %s", s)
        continue
    data = r.json()
    data.pop('Meta Data')
    df = pd.DataFrame(data['Monthly Time Series']).T
    df['Symbol'] = s
    if i == 0:
        df.to_csv('monthly_prices.csv')
    else:
        df.to_csv('monthly_prices.csv', mode = 'a', header = False)
    sleep(15)